In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials as yfs
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# !pip install yahoofinancials

In [2]:
def start(ticker):
  
    yahoo_financials = yfs(ticker)

    cash = yahoo_financials.get_financial_stmts('annual','cash')
    income = yahoo_financials.get_financial_stmts('annual','income')
    balance = yahoo_financials.get_financial_stmts('annual','balance')
    stock = yahoo_financials.get_key_statistics_data()[ticker]

    cash_balance = Sheet_Data(ticker,cash)
    income_balance = Sheet_Data(ticker,income)
    balance_balance = Sheet_Data(ticker,balance)
    print(ticker)
    return DATA(cash_balance,income_balance,balance_balance,stock,yahoo_financials)

In [3]:
def Sheet_Data(ticker,sheet_data):
    save = []
    
    df = pd.DataFrame.from_dict({(i,j): sheet_data[i][j] 
                            for i in sheet_data.keys() 
                            for j in sheet_data[i].keys()},
                        orient='index')

    for i in df.columns:
        f = len(df.columns) - i - 1
        df2 = pd.json_normalize(df[f],max_level = 1)

        #Gets  Date and renames 
        Date = df2.columns[0]
        Date = Date[0:11]
        df2.index = df2.index.rename(ticker)
        df2.columns = df2.columns.str.replace(str(Date),'')

        #Tranposes DataFrame and changes it to the 
        df2 = df2.T
        df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:4]))},inplace = True)
        #display(df2)
        save.append(df2)
    df3 = pd.concat(save,axis = 1)
    #display(df3)
    #filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
    #print(filename)
    #df3.to_csv('{}.csv'.format(filename))
    return df3
#filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
#df3.t

In [4]:
#WACC = Wd * Rd * (1 - t) + We + Re
def DATA(cash_balance,income_balance,balance_balance,stock,yahoo_financials ):
    INCOME2 = income_balance.loc[['incomeBeforeTax','incomeTaxExpense']]
    t = income_balance.loc['incomeBeforeTax']/ income_balance.loc['incomeTaxExpense']
    t = t[-1]/ 100 



    #yahoo_financials.get_beta()
    df = pd.read_html('https://finance.yahoo.com/bonds')
    Rf = float(df[0]['Last Price'][2])
    try: 
        beta = float(yahoo_financials.get_beta())
    except:
        beta = 0
    Rm = float(10.0)
    Ra = Rf + beta * (Rm - Rf)
    #print(Ra)

    try:
        Balance2 = balance_balance.loc[['shortLongTermDebt','longTermDebt']]#.astype#(float)
    except:
        Balance2 = balance_balance.loc[['longTermDebt']]

    Total_debt = Balance2.sum()[-1] 
    Balance2 =   -(income_balance.loc['interestExpense'][-1]) / Total_debt
    Rd = (1-Balance2) * t #  Rd * (1 - t)
    #Balance2.sum()


    Market_Cap = yahoo_financials.get_market_cap() # MarketCap
    Total =  Total_debt + Market_Cap 
    Wd = Total_debt/ Total 
    We = Market_Cap/ Total 
    Re = Ra
    Required_Rate = (Wd * Rd + We * Re)/100
    Required_Rate

    FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netIncome']] # how to select rows 
    Revenue = income_balance.loc[['totalRevenue','netIncome']]
    Growth_Rate = Revenue.pct_change().mean()
    #FCF is CFO - CAPEX = FCF
    #FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netBorrowings']] # More 

    df = pd.DataFrame()
    df['FCF'] = FCF.loc['totalCashFromOperatingActivities'] + FCF.loc['capitalExpenditures']
    df['Net Income'] = FCF.loc['netIncome']
    #df['FCF/Net Income'] = df['FCF']/df['Net Income']
    minimum = (df['FCF']/df['Net Income']).min()
    #print(Wd)
    #print(Rd)
    #print(We)
    #print(Re)

    #print(Ra)
    #print(Rf)
    #print(Required_Rate)
    return  DCF_VALUE(Required_Rate,minimum, df,stock), DCF_VALUE_CON(df,stock)

# Anaylsis

In [5]:
def DCF_VALUE(Required_Rate,minimum, df,stock):
    outstanding_shares = stock['sharesOutstanding']

    Required_rate = Required_Rate
    Pre_rate = .025
    Cashflowgrowthrate = minimum

    years = [1,2,3,4]
    freecash_flow = df['FCF'].values

    terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

    futurefreecashflow = []
    discountfactor = []
    discountfuturefreecashflow = []
    for  year in years:
        cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
        futurefreecashflow.append(cashflow)
        discountfactor.append((1+ Required_rate)**year)


    for i in range(0,len(years)):
        discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

    #print(discountfuturefreecashflow)
    dis = terminalvalue/(1+Required_rate)**len(years)
    discountfuturefreecashflow.append(dis)
    value = sum(discountfuturefreecashflow)
    fairvalue = value/outstanding_shares

    
    print(ticker)
    print(fairvalue)
    #print(yahoo_financials.get_current_price())
    #tickers.append(ticker)
    #Value.append(fairvalue)
    #print(Cashflowgrowthrate)
    #print(Required_Rate)
    #return fairvalue
#print(stock.get_current_price())
#DCF_VALUE()

In [6]:
def DCF_VALUE_CON(df,stock):
    outstanding_shares = stock['sharesOutstanding']

    Required_rate = .075
    Pre_rate = .025
    Cashflowgrowthrate = .035

    years = [1,2,3,4]
    freecash_flow = df['FCF'].values

    terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

    futurefreecashflow = []
    discountfactor = []
    discountfuturefreecashflow = []
    for  year in years:
        cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
        futurefreecashflow.append(cashflow)
        discountfactor.append((1+ Required_rate)**year)


    for i in range(0,len(years)):
        discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

    #print(discountfuturefreecashflow)
    dis = terminalvalue/(1+Required_rate)**len(years)
    discountfuturefreecashflow.append(dis)
    value = sum(discountfuturefreecashflow)
    fairvalue = value/outstanding_shares

    print(ticker)
    print(fairvalue)
    #print(yahoo_financials.get_current_price())

    #print(Cashflowgrowthrate)
    #print(Required_rate)
    #tickers.append(ticker)
    #ValueCon.append( fairvalue)
#print(stock.get_current_price())
#DCF_VALUE_CON()

In [7]:
def Collect(sdsd):
    global ticker,maindf,tickers,Value,ValueCon
    maindf = pd.DataFrame()
    tickers = []
    Value  = []
    ValueCon = []
    #sdsd = ['AMZN','SQ','GOOG','MSFT']
    try:
        for ticker in sdsd:
            %time start(ticker)
    except:
            print('This Failed ' + str(ticker))

In [8]:
df = pd.read_csv('FINALTICKER.csv')
ttt = 'QQQ'
df = df[df['ETF'].str.contains('{}'.format(ttt))].reset_index(drop = 'True')
df = df[~df['ETF'].str.contains('QQQJ')].reset_index(drop = 'True')
df
#sdsd = df['Ticker'][0:5]
#Collect(sdsd)

,Unnamed: 0,Ticker,Company,Sector,Industry,P/E,EPS (ttm),Insider Own %,Perf Week %,Market Cap,...,Death MACD,Golden MACD,50-MA,200-MA,50 %,200 %,MOM-25,EMOM-25,ETF,Tick
0,0,AAPL,Apple Inc.,Technology,Consumer Electronics,32.83,3.70,0.07,0.13,2.033870e+12,...,2021-02-01,2021-01-22,131.402606,113.612943,-7.596962,6.871627,97.866,100.522,"ARKF,ARKQ,QQQ,PTF",AAPL
1,8,ADBE,Adobe Inc.,Technology,Software - Infrastructure,40.58,10.86,0.10,-4.10,2.107900e+11,...,2021-02-22,2021-02-02,477.912399,464.048001,-7.759246,-5.003362,99.426,99.491,"ARKW,QQQ,PTF,PSJ,XSW",ADBE
2,9,ADI,"Analog Devices, Inc.",Technology,Semiconductors,39.44,3.77,0.10,-4.45,5.417000e+10,...,2021-02-25,2021-02-11,152.787585,129.248763,-2.557528,15.188723,101.736,104.142,QQQ,ADI
3,10,ADP,"Automatic Data Processing, Inc.",Industrials,Staffing & Employment Services,30.99,5.75,0.10,2.44,7.695000e+10,...,2020-11-20,2021-01-28,169.381800,154.007302,5.241531,15.747758,101.412,102.120,"QQQ,XSW",ADP
4,13,ADSK,"Autodesk, Inc.",Technology,Software - Application,49.22,5.43,0.10,-3.12,6.051000e+10,...,2021-02-23,2021-02-08,297.690999,256.793600,-10.178675,4.126427,96.263,100.694,"ARKQ,QQQ,XSW",ADSK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,435,WBA,"Walgreens Boots Alliance, Inc.",Healthcare,Pharmaceutical Retailers,0.00,-0.81,0.30,-1.52,4.107000e+10,...,2021-02-02,2021-01-06,46.649770,40.642467,1.179491,16.134682,110.257,108.534,QQQ,WBA
99,436,WDAY,"Workday, Inc.",Technology,Software - Application,0.00,-1.20,0.70,-3.33,5.828000e+10,...,2021-02-23,2021-02-01,245.730600,214.035000,-3.544776,10.738898,111.850,108.395,"ARKF,QQQ,XSW",WDAY
100,445,XEL,Xcel Energy Inc.,Utilities,Utilities - Regulated Electric,22.00,2.79,0.30,4.71,3.369000e+10,...,2021-01-29,2021-03-05,63.226200,66.386054,-2.967441,-7.586012,94.441,95.115,QQQ,XEL
101,446,XLNX,"Xilinx, Inc.",Technology,Semiconductors,48.00,2.50,0.10,-7.94,2.837000e+10,...,2021-02-22,2021-02-08,138.271401,117.224960,-13.250318,2.324624,94.569,98.072,"ARKQ,QQQ",XLNX


global ticker,maindf,tickers,Value,ValueCon
maindf = pd.DataFrame()
tickers = []
Value  = []
ValueCon = []
ticker = 'AAPL'
%time start(ticker)

In [12]:
tickers = ['ADBE','MSFT']
for ticker in tickers: 
    try:
        %time start(ticker)
    except:
        0

ADBE
ADBE
363.7922505917927
ADBE
210.31415275604724
Wall time: 25.9 s
MSFT
MSFT
213.08984491606617
MSFT
113.62739473346839
Wall time: 31.9 s


In [10]:
ticker = 'LMT'
%time start(ticker)

NameError: name 'Value' is not defined

In [137]:
ValueCon

[82.99576991597596, 180.61713617426543, 437.2652428087806]

df = pd.concat([df2[2],df2[3],df2[4]])
#display(df)
x = []
x.append(df['Unnamed: 0'].dropna())
x.append(df['Liabilities & Shareholders Equity'].dropna())
x.append(df['Shareholders Equity'].dropna())

y = pd.concat(x).tolist()
df = df.drop(['Liabilities & Shareholders Equity','Shareholders Equity','Unnamed: 0'], axis=1)
df['Liabilities & Shareholders Equity'] = y
df.set_index('Liabilities & Shareholders Equity')

x#Anaylist esitmates 
df = pd.read_html('https://finance.yahoo.com/quote/AAPL/analysis?p=AAPL')
df = df[1] 
df.index = df[df.columns[0]]
df1 = df.index 
df1 = df[df.columns[3:5]]
df1 = df1.drop(['Sales Growth (year/est)'])

for i in df1.columns:
    df1[i] = df1[i].str.replace('B','e9')
    df1[i] = df1[i].astype(float)

#df1.loc['Avg. Estimate']
df1.loc['Avg. Estimate'].tolist()

Revenue1 = Revenue.loc['totalRevenue']
strs = int(Revenue.columns[3])
Revenue1['{}'.format(strs+1)] = df1.loc['Avg. Estimate'][0]
Revenue1['{}'.format(strs+2)] = df1.loc['Avg. Estimate'][1]
growth = 1 + Revenue.T.pct_change().sum()/5

Revenue1['{}'.format(strs+3)] = l.drop_index() * growth
Revenue1['{}'.format(strs+4)] = l2.drop_index() * growth
Revenue.loc['netIncome']/Revenue.loc['totalRevenue']

ticker = 'LMT'
yahoo_financials = yfs(ticker)

cash = yahoo_financials.get_financial_stmts('annual','cash')
income = yahoo_financials.get_financial_stmts('annual','income')
balance = yahoo_financials.get_financial_stmts('annual','balance')
stock = yahoo_financials.get_key_statistics_data()[ticker]

sample = pd.DataFrame()
sample['Free Cash Flow'] = df['FCF']
sample['Cash Flow'] = futurefreecashflow
sample['discountfactor'] = discountfactor
sample

df = pd.DataFrame.from_dict({(i): stock[i] 
                            for i in stock.keys() 
                            },
                        orient='index')
df

aapl.info['sharesOutstanding']
stock.keys()

pd.DataFrame.from_dict({(i): stock[i] 
for i in stock.keys()},orient='index')

# Testing for working all of the files work

df = pd.DataFrame.from_dict({(i,j): balance_sheet_data[i][j] 
                           for i in balance_sheet_data.keys() 
                           for j in balance_sheet_data[i].keys()},
                       orient='index')
for i in df.columns:
    df[i].tolist()
    df2 = pd.json_normalize(df[i],max_level = 1)

    #Gets  Date and renames 
    Date = df2.columns[0]
    Date = Date[0:11]
    df2.index = df2.index.rename(ticker)
    df2.columns = df2.columns.str.replace(str(Date),'')

    #Tranposes DataFrame and changes it to the 
    df2 = df2.T
    df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:10]))},inplace = True)
    #display(df2)
    save.append(df2)
df3 = pd.concat(save,axis = 1)
display(df3)